## Lorenz System with Missing Variables

In [ ]:
# --> Simulating the Lorenz system.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from Lorenz import Lorenz

# --> Sets the parameters to their classical values.
sigma, rho, beta = 10., 28., 8./3.

# --> Integration time.
t = np.linspace(0, 20, 2000)

# --> Produce the date to be used in the sparse identification.
x0 = np.array([-8., 7., 27.]) # Initial condition.
x, dx = Lorenz(x0, sigma, rho, beta, t)

# --> Slightly different initial condition to highlight the chaotic nature.
y0 = np.array([-8.01, 7., 27.]) # Initial condition.
y, dy = Lorenz(y0, sigma, rho, beta, t)

In [ ]:
# --> Plot time traces of the two trajectories as well as the corresponding stange attractor.
w = 10
fig = plt.figure(figsize=(1.5*w, w/2))
gs = GridSpec(3, 6)

ax0 = fig.add_subplot(gs[0, :3])
ax0.plot(t, x[:, 0])
ax0.plot(t, y[:, 0])
ax0.set_ylabel('x')
ax0.set_xticks([])
ax0.set_xlim(0, 20)

ax1 = fig.add_subplot(gs[1, :3])
ax1.plot(t, x[:, 1])
ax1.plot(t, y[:, 1])
ax1.set_ylabel('y')
ax1.set_xticks([])
ax1.set_xlim(0, 20)

ax2 = fig.add_subplot(gs[2, :3])
ax2.plot(t, x[:, 2])
ax2.plot(t, y[:, 2])
ax2.set_ylabel('z')
ax2.set_xlabel('t')
ax2.set_xlim(0, 20)

ax3 = fig.add_subplot(gs[:, 3:], projection='3d')
ax3.plot(x[:, 0], x[:, 1], x[:, 2])
ax3.plot(y[:, 0], y[:, 1], y[:, 2])
ax3.set_xlabel('x', labelpad=10)
ax3.set_ylabel('y')
ax3.set_zlabel('z')
plt.show()

### Original Algorithm

In [ ]:
# --> Creation of the library Theta.
from sklearn.preprocessing import PolynomialFeatures
library = PolynomialFeatures(degree=2, include_bias=True)
Theta = library.fit_transform(x)
n_lib = library.n_output_features_

from scipy.linalg import block_diag
A = block_diag(Theta, Theta, Theta)
b = dx.flatten(order='F')

In [ ]:
# --> Sequentially hard-thresholded estimator.
from sparse_identification import sindy
from scipy.linalg import block_diag

shols = sindy(l1=0.01, solver='lstsq')

# --> Fit the OLS model.
shols.fit(A, b)
print('Total number of possible terms :', shols.coef_.size)
print('Number of non-zero coefficients :', np.count_nonzero(shols.coef_))

In [ ]:
print("Identified equation for x : \n")
print(shols.coef_[:n_lib], "\n")
print("\n Identified equation for y : \n")
print(shols.coef_[n_lib:2*n_lib], "\n")
print("\n Identified equation for y : \n")
print(shols.coef_[2*n_lib:3*n_lib], "\n")

In [ ]:
# Case where one of the variables is missing
x_trunc = x[:, :2]
dx_trunc = x[:, :2]

from sklearn.preprocessing import PolynomialFeatures
library = PolynomialFeatures(degree=5, include_bias=True)
Theta = library.fit_transform(x_trunc)
n_lib = library.n_output_features_

from scipy.linalg import block_diag
A_trunc = block_diag(Theta, Theta)
b_trunc = dx_trunc.flatten(order='F')

shols_trunc = sindy(l1=0.01, solver='lstsq')

# --> Fit the OLS model to truncated data
shols_trunc.fit(A_trunc, b_trunc)
print('Total number of possible terms :', shols_trunc.coef_.size)
print('Number of non-zero coefficients :', np.count_nonzero(shols_trunc.coef_))

In [ ]:
print("Identified equation for x : \n")
print(shols.coef_[:n_lib], "\n")
print("\n Identified equation for y : \n")
print(shols.coef_[n_lib:2*n_lib], "\n")

### New Algorithm for Missing Variables applied to Lotka-Volterra System

We have the Lotka-Volterra system
\begin{align*}
\dot{x} &= x(1 - y) \\
\dot{y} &= y(-1 + x - z) \\
\dot{z} &= z(-1 + y)
\end{align*}
with initial conditions $[x(0), y(0), z(0)] = [0.5, 1, 2]$

In [35]:
import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from Lotka_Volterra import Lotka_Volterra

alpha = np.array([1, -1, -1])
beta = np.array([[0, -1, 0], [1, 0, -1], [0, 1, 0]])
t = np.linspace(0, 5, 2000)
trajectories = []
total = 0
while total < 20:
    x0 = np.random.uniform(-2, 2, 3) # Initial condition.
    x, dx = Lotka_Volterra(x0, alpha, beta, t)
    if np.max(np.abs(x)) > 10:
        continue
    trajectories.append(x)
    total += 1
trajectories = np.array(trajectories)

/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [38]:
# Case where one of the variables is missing
x_trunc = trajectories[:, :, :2]
dt = t[1] - t[0]
l1 = 0.01
degree = 2
num_coeffs = 10 # Each Lotka-Volterra system has at most four nonzero coefficients

In [39]:
from pprint import pprint
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy.linalg import block_diag
from sklearn.preprocessing import PolynomialFeatures
from sparse_identification.utils import derivative

num_trajectories = trajectories.shape[0]
x0 = [1e-3] * (3*num_coeffs) # initialize coefficients to be small values 
l2 = 0.01
iter_num = 0

def make_terms(y):
    # make the polynomial terms of the Lotka-Volterra system
    terms = []
    base = np.concatenate(([1], y))
    for i in range(len(y)):
        terms.append(y[i] * base)
    return np.array(terms)
'''
def system_vals(y0, t, coeffs):
    terms = make_terms(y0)
    A = block_diag(*terms)
    return A @ coeffs
'''
def system_vals(y0, t, coeffs):
    Theta = library.fit_transform([y0])
    A = block_diag(Theta, Theta, Theta)
    return A @ coeffs

def f(y):
    global iter_num
    diff_sq = 0
    for ind, x_slice_trunc in enumerate(x_trunc):
        coeffs = y[:3*num_coeffs]
        #z_start = y[3*num_coeffs+ind]
        #initial = [x_slice_trunc[0, 0], x_slice_trunc[0, 1], z_start]
        initial = trajectories[ind][0]
        x_est = odeint(system_vals, initial, t, args=(coeffs,))
        x_est = x_est[:, :2] # only keep first two columns so we can compare with actual values
        diff_sq += np.sum((x_slice_trunc - x_est) ** 2)
    if iter_num % 100 == 0:
        print('On iteration {}, residual is {}'.format(iter_num, diff_sq))
    iter_num += 1
    
    return diff_sq

for _ in range(5):
    res = minimize(f, x0)
    print('Result after this step:')
    pprint(res.x)
    coeffs = res.x
    for i in range(3):
        coeff_block = coeffs[i*num_coeffs:(i+1)*num_coeffs]
        xmax = abs(coeff_block[np.nonzero(coeff_block)]).mean()
        to_remove = [k for k in range(len(coeff_block)) if abs(coeff_block[k]) < l1*xmax]
        for k in to_remove:
            coeffs[i*num_coeffs + k] = 0
    x0 = coeffs
    iter_num = 0
    print('Restarting process after removing extraneous coefficients...')

On iteration 0, residual is 93320.22045130731


/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


On iteration 100, residual is 93320.22105199778


/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


On iteration 200, residual is 72273.71699128239
On iteration 300, residual is 54565.95445338145
On iteration 400, residual is 46794.550546258855
On iteration 500, residual is 42660.61319369686
On iteration 600, residual is 39533.667433413895
On iteration 700, residual is 34491.56243730589


...

Result after this step:
array([ 0.        ,  0.99999943,  0.        ,  0.        ,  0.        ,
       -1.00000185,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -1.00000563,  0.        ,  0.        ,
        0.99998975,  0.        ,  0.        , -1.00000074,  0.        ,
        0.        ,  0.        ,  0.        , -0.99997551,  0.        ,
        0.        ,  0.        ,  0.        ,  0.99997369,  0.        ])
Restarting process after removing extraneous coefficients...
On iteration 0, residual is 6.407131398527913e-05


/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


On iteration 100, residual is 6.405028892013598e-05


/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Result after this step:
array([ 0.        ,  0.99999943,  0.        ,  0.        ,  0.        ,
       -1.00000185,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -1.00000563,  0.        ,  0.        ,
        0.99998975,  0.        ,  0.        , -1.00000074,  0.        ,
        0.        ,  0.        ,  0.        , -0.99997551,  0.        ,
        0.        ,  0.        ,  0.        ,  0.99997369,  0.        ])
Restarting process after removing extraneous coefficients...


In [ ]:
alpha2 = np.array([0.9982, -0.9774, -0.9531])
beta2 = np.array([[0, -1.00015232, 0.00813716], [0.99634992, 0, -7.23730658], [-0.01271289, 0.99328139, -0.13339578]])
t = np.linspace(0, 5, 2000)
x0_2 = np.array([0.5, 1, 2.79314641e-01]) # Initial condition.
x2, dx2 = Lotka_Volterra(x0_2, alpha2, beta2, t)
np.max(x2 - x)

If we remove the variable $z$ ad use the above algorithm, we get the system
\begin{align*}
\dot{x} &= x(9.98188683 \cdot 10^{-1} + 5.02546447 \cdot 10^{-4}x - 1.00015232y + 8.13716153 \cdot 10^{-3}z) \\
\dot{y} &= y(-9.77408801 \cdot 10^{-1} + 9.96349918 \cdot 10^{-1}x - 1.99183904 \cdot 10^{-3}y - 7.23730658z) \\
\dot{z} &= z(-9.53057554 \cdot 10^{-2} -1.27128901 \cdot 10^{-2}x + 9.93281392 \cdot 10^{-1}y  -1.33395777 \cdot 10^{-1}z)
\end{align*}

## Case where points are not uniformly spaced.

In [12]:
# Method that uses scipy.integrate.odeint to approximate trajectories given coefficients

import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import PolynomialFeatures
from Lotka_Volterra import Lotka_Volterra

alpha = np.array([1, -1, -1])
beta = np.array([[0, -1, 0], [1, 0, -1], [0, 1, 0]])
# choose points that are not uniformly spaced
t = np.sort(np.random.uniform(0, 5, 2000))
trajectories = []
total = 0
while total < 10:
    x0 = np.random.uniform(-2, 2, 3) # Initial condition.
    x, dx = Lotka_Volterra(x0, alpha, beta, t)
    if np.max(np.abs(x)) > 10:
        continue
    trajectories.append(x)
    total += 1
trajectories = np.array(trajectories)
l1 = 0.01
degree = 2
num_coeffs = 10
library = PolynomialFeatures(degree=degree, include_bias=True)

x0 = [1e-3] * (3*num_coeffs) # initialize coefficients to be small values 
l2 = 0.01
iter_num = 0

def system_vals(y0, t, coeffs):
    Theta = library.fit_transform([y0])
    A = block_diag(Theta, Theta, Theta)
    return A @ coeffs

def f(y):
    global iter_num
    diff_sq = 0
    for ind, trajectory in enumerate(trajectories):
        initial = trajectory[0]
        x_est = odeint(system_vals, initial, t, args=(y,))
        diff_sq += np.sum((trajectory - x_est) ** 2)
    if iter_num % 100 == 0:
        print('On iteration {}, residual is {}'.format(iter_num, diff_sq))
    iter_num += 1
    
    return diff_sq

for _ in range(5):
    res = minimize(f, x0, tol=1e-5)
    print('Result after this step:')
    pprint(res.x)
    coeffs = res.x
    for i in range(3):
        coeff_block = coeffs[i*num_coeffs:(i+1)*num_coeffs]
        xmax = abs(coeff_block[np.nonzero(coeff_block)]).mean()
        to_remove = [k for k in range(len(coeff_block)) if abs(coeff_block[k]) < l1*xmax]
        for k in to_remove:
            coeffs[i*num_coeffs + k] = 0
    x0 = coeffs
    iter_num = 0
    print('Restarting process after removing extraneous coefficients...')

/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


On iteration 0, residual is 80390.52483089681
On iteration 100, residual is 59021.27863142977
On iteration 200, residual is 53019.77766867208
On iteration 300, residual is 94115.74716305545
On iteration 400, residual is 43395.28664164103
On iteration 500, residual is 40842.09390822385


/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


On iteration 600, residual is 40038.96721981872
On iteration 700, residual is 37045.25706309827
On iteration 800, residual is 36065.94460267344
On iteration 900, residual is 34130.89384199084
On iteration 1000, residual is 32938.17158151143
On iteration 1100, residual is 40917.84669431624


/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


...

Result after this step:
array([ 0.        ,  1.00000686,  0.        ,  0.        ,  0.        ,
       -1.00000972,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -1.00003163,  0.        ,  0.        ,
        1.00000499,  0.        ,  0.        , -1.00000003,  0.        ,
        0.        ,  0.        ,  0.        , -0.99998445,  0.        ,
        0.        ,  0.        ,  0.        ,  1.0000026 ,  0.        ])
Restarting process after removing extraneous coefficients...
On iteration 0, residual is 0.00011460802136964002


/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/home/alexander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: overflow encountered in square
/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:628: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]


On iteration 100, residual is 0.00011472408593958927


/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Result after this step:
array([ 0.        ,  1.00000686,  0.        ,  0.        ,  0.        ,
       -1.00000972,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -1.00003163,  0.        ,  0.        ,
        1.00000499,  0.        ,  0.        , -1.00000003,  0.        ,
        0.        ,  0.        ,  0.        , -0.99998445,  0.        ,
        0.        ,  0.        ,  0.        ,  1.0000026 ,  0.        ])
Restarting process after removing extraneous coefficients...


In [36]:
import numpy as np
from cvxopt import solvers, matrix, spmatrix
from scipy import sparse
from scipy.linalg import norm

def approx_derivative(x, t):
    # compute differences in x
    dx = np.zeros_like(x)
    dx[1:-1, :] = (x[2:, :] - x[:-2, :])
    dx[0,:]    = (x[1,:] - x[0,:])
    dx[-1,:]   = (x[-1,:] - x[-2,:])
    # compute differences in t
    dt = np.zeros_like(t)
    dt[1:-1] = (t[2:] - t[:-2])
    dt[0]    = (t[1] - t[0])
    dt[-1]   = (t[-1] - t[-2])
    
    return (dx/dt[:, None]).flatten(order='F')

def numpy_to_cvxopt_matrix(A):
    '''
    This function takes as input a numpy/SciPy array/matrix and converts it to a CVX
     format.

    Inputs:
    ------
        A : NumPy/SciPy array or matrix.

    Outputs:
    -------
        A : Corresponding matrix in CVX format.
    '''

    # --> None case.
    if A is None:
        return None

    # --> sparse SciPy case.
    if sparse.issparse(A):
        if isinstance(A, sparse.spmatrix):
            return scipy_sparse_to_spmatrix(A)
        else:
            return A
    else:
    # --> Dense matrix or NumPy array.
        if isinstance(A, np.ndarray):
            if A.ndim == 1:
                return matrix(A, (A.shape[0], 1), 'd')
            else:
                return matrix(A, A.shape, 'd')
        else:
            return A

In [40]:
# Method that uses derivative approximation 
# Does not appear to work very well 
from cvxopt import solvers
import numpy as np
np.random.seed(1337)
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import least_squares
from scipy.linalg import block_diag
from scipy.sparse import csr_matrix
from sklearn.preprocessing import PolynomialFeatures
from Lotka_Volterra import Lotka_Volterra
from sparse_identification.solvers import hard_threshold_lstsq_solve
from sparse_identification.utils import derivative

alpha = np.array([1, -1, -1])
beta = np.array([[0, -1, 0], [1, 0, -1], [0, 1, 0]])
# choose points that are not uniformly spaced
t = np.sort(np.random.uniform(0, 5, 2000))
trajectories = []
total = 0
while total < 5:
    x0 = np.random.uniform(-2, 2, 3) # Initial condition.
    x, dx = Lotka_Volterra(x0, alpha, beta, t)
    if np.max(np.abs(x)) > 10:
        continue
    trajectories.append(x)
    total += 1
trajectories = np.array(trajectories)
l1 = 0.01
degree = 2
num_coeffs = 10
library = PolynomialFeatures(degree=degree, include_bias=True)

x0 = [1e-3] * (3*num_coeffs) # initialize coefficients to be small values 
l2 = 0.01
iter_num = 0

def minimize_diff(x0):
    global iter_num
    diff_sq = 0
    A, b = [], []
    for ind, trajectory in enumerate(trajectories):
        deriv_est_lhs = approx_derivative(trajectory, t) # approximated derivates in LHS of system 
        b.append(deriv_est_lhs)
        Theta = library.fit_transform(trajectory)
        A_traj = block_diag(Theta, Theta, Theta)
        A.append(A_traj)
    A = np.concatenate(A)
    b = np.concatenate(b)
    coeffs = hard_threshold_lstsq_solve(A, b)
    #coeffs = least_squares(lambda x: A @ x - b, x0).x
    return coeffs

for _ in range(5):
    coeffs = minimize_diff(x0)
    for i in range(3):
        coeff_block = coeffs[i*num_coeffs:(i+1)*num_coeffs]
        xmax = abs(coeff_block[np.nonzero(coeff_block)]).mean()
        to_remove = [k for k in range(len(coeff_block)) if abs(coeff_block[k]) < l1*xmax]
        for k in to_remove:
            coeffs[i*num_coeffs + k] = 0
    x0 = coeffs
    print('After iteration {} we have the coefficients'.format(_))
    print(x0)

/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/home/alexander/anaconda3/lib/python3.6/site-packages/cvxopt/coneprog.py:4329: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if solver == 'mosek':


After iteration 0 we have the coefficients
[ 0.          0.99998387  0.          0.          0.         -0.99997357
  0.          0.          0.          0.          0.          0.
 -0.99997142  0.          0.          0.99996688  0.          0.
 -0.99996966  0.          0.          0.          0.         -0.99999252
  0.          0.          0.          0.          0.99999122  0.        ]
After iteration 1 we have the coefficients
[ 0.          0.99998387  0.          0.          0.         -0.99997357
  0.          0.          0.          0.          0.          0.
 -0.99997142  0.          0.          0.99996688  0.          0.
 -0.99996966  0.          0.          0.          0.         -0.99999252
  0.          0.          0.          0.          0.99999122  0.        ]
After iteration 2 we have the coefficients
[ 0.          0.99998387  0.          0.          0.         -0.99997357
  0.          0.          0.          0.          0.          0.
 -0.99997142  0.          0.     

In [6]:
import numpy as np
from sparse_identification import sindy
from scipy.linalg import block_diag
from sklearn.preprocessing import PolynomialFeatures
from Lotka_Volterra import Lotka_Volterra

alpha = np.array([1, -1, -1])
beta = np.array([[0, -1, 0], [1, 0, -1], [0, 1, 0]])
t = np.sort(np.random.uniform(0, 5, 2000))
total = 0
while total < 1:
    x0 = np.random.uniform(-2, 2, 3) # Initial condition.
    x, dx = Lotka_Volterra(x0, alpha, beta, t)
    if np.max(np.abs(x)) > 10:
        continue
    total += 1

library = PolynomialFeatures(degree=2, include_bias=True)
Theta = library.fit_transform(x)
n_lib = library.n_output_features_
A = block_diag(Theta, Theta, Theta)
b = dx.flatten(order='F')

shols = sindy(l1=0.01, solver='lstsq')
shols.fit(A, b)
print(shols.coef_)

[  2.43042270e+01   2.47919722e+01   5.94024088e-01  -7.10533910e+00
   4.79815222e+00   2.80948158e+02   1.85550894e+00   2.84196219e+01
   3.38050378e+01   7.94691683e+00   0.00000000e+00   0.00000000e+00
  -3.55264623e+00  -1.24889573e+00   0.00000000e+00   8.82742666e+00
   0.00000000e+00   1.79312658e-01  -3.17899126e+00  -1.60693014e+00
   0.00000000e+00   0.00000000e+00  -6.77502276e-01  -3.27582337e+00
   0.00000000e+00  -3.84571483e+00   0.00000000e+00   0.00000000e+00
   4.22210757e+00   2.19825519e+00]


/home/alexander/anaconda3/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/home/alexander/anaconda3/lib/python3.6/site-packages/cvxopt/coneprog.py:4329: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if solver == 'mosek':
